In [ ]:
!pip install requests


In [ ]:
import requests
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
import numpy as np

def get_food_nutrition(api_key, food_name):
    base_url = "https://api.nal.usda.gov/fdc/v1/foods/search"

    # Set up parameters for the API request
    params = {
        'query': food_name,
        'api_key': api_key,
    }

    # Make the API request
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Check if any foods were found
        if 'foods' in data and data['foods']:
            # Get the first food item
            food_item = data['foods'][0]

            # Get the calorie content
            if 'foodNutrients' in food_item:
                for nutrient in food_item['foodNutrients']:
                    if nutrient['nutrientName'] == 'Energy':
                        return nutrient['value']

    # If the request was not successful or no data was found, return None
    return None

def predict_food(img_path):
    img = image.load_img(img_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Load InceptionV3 model pre-trained on ImageNet data
    model_incep = InceptionV3(weights='imagenet')

    predictions = model_incep.predict(img_array)
    decoded_predictions = decode_predictions(predictions, top=1)[0]

    return decoded_predictions[0][1]

# Replace 'your_api_key' with your actual FoodData Central API key
api_key = 'fjFLVVfbL18IC3vwchQJ7OJiY5T71E6yrHgmYd5Q'
img_path = 'pizza.jpeg'  # Replace with the path to your food image

# Predict the food item from the image
predicted_food = predict_food(img_path)

if predicted_food:
    print(f"Predicted Food: {predicted_food}")

    # Get calorie information using FoodData Central API
    calories = get_food_nutrition(api_key, predicted_food)

    if calories is not None:
        print(f"The estimated calories in {predicted_food} are: {calories} kcal")
    else:
        print(f"No data found for {predicted_food}")
else:
    print("Unable to predict food from the image.")


1/1 [==============================] - 2s 2s/step
Predicted Food: pizza
The estimated calories in pizza are: 250 kcal
